# Лабораторная работа №6

## Библиотека pandas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os

**1**.	Таблица moscow-buildings.csv содержит информацию о годах постройки жилых домов в Москве.

1)	Загрузите базу данных moscow-buildings.csv и сохраните ее как df_build. Удалите из базы данных строки, в которых значения года постройки не указаны («н.д.»), а после превратите столбец house_year в числовой.

2)	Посмотрите на минимальное и максимальное значения года постройки. Разумны ли они? Если нет, уберите из базы данных «странные» значения года постройки. Подсчитайте количество удаленных записей.

3)	Выберите из базы данных строки, соответствующие Басманному району и сохраните их в базу basm_data.

4)	Сгруппируйте данные в базе по районам и посмотрите, сколько домов в базе относятся к каждому району.

5)	Найдите средний возраст дома (возраст – в годах, считаем от текущего года) по каждому району. Подсказка: сначала добавьте переменную house_age («возраст дома»).

6)	Найдите самый старый дом и выведите информацию о нем.

In [3]:
directory = "task_1"

if not os.path.isdir(directory):
    os.makedirs(directory)

#1:
df_build = pd.read_csv("moscow-buildings.csv")
df_build = df_build[df_build["house_year"] != "н.д."]
df_build["house_year"] = df_build["house_year"].astype(int)

#2:
first_building_year = 1357
current_year = datetime.now().year

size_before_deleting = df_build.shape[0]
df_build = df_build[(df_build["house_year"] >= first_building_year) &  (df_build["house_year"] <= current_year)]
size_after_deleting = df_build.shape[0]

print(f"deleted rows count: {size_before_deleting - size_after_deleting}")
df_build.to_csv(f"{directory}/df_build.csv")

#3:
basm_data = df_build[df_build["area_name"].str.contains("Басманный")]
basm_data.to_csv(f"{directory}/basm_data.csv")

#4:
def count(x):
    return len(x)

areas_buildings_counts: pd.Series = df_build.groupby("area_name").agg(count).iloc[:, 0]
areas_buildings_counts.rename("buildings_count").to_csv(f"{directory}/areas_buildings_counts.csv")

#5:
df_build["house_age"] = current_year - df_build["house_year"]
areas_mean_buildings_ages: pd.Series = df_build.groupby("area_name").agg("mean")["house_age"]
areas_mean_buildings_ages.rename("mean_building_age").to_csv(f"{directory}/areas_mean_buildings_ages.csv")

#6:
max_age = df_build[["house_age"]].agg("max")[0]
df_build[df_build["house_age"] == max_age]

deleted rows count: 24


,area_id,full_address,city,address,street_prefix,street_name,building,house_id,area_name,house_year,house_age
7304,2281064,г Москва пр-кт 40 лет Октября д.12/10,Москва,пр-кт 40 лет Октября д.12/10,пр-кт,40 лет Октября,д.12/10,7566338,муниципальный округ Люблино,1659,364


**2.**	Загрузите таблицу titanic.csv и найдите ответы на следующие вопросы. Если ответ нужно приводить в процентах, то это должно быть число в интервале от 0 до 100, округленное до двух знаков, знак процента не нужен.  

1)	Посчитайте долю выживших пассажиров.

2)	Какое количество мужчин и женщин ехало на корабле? Определите долю выживших женщин и выживших мужчин.

3)	Какую долю пассажиры первого, второго и третьего классов составляли среди всех пассажиров? Определите долю выживших пассажиров каждого класса.

4)	 Посчитайте среднее и медиану возраста пассажиров. Посчитайте среднее и медиану возраста выживших и невыживших пассажиров.

5)	Добавьте столбцы FirstNameList с именем пассажира. Обратите внимание, что встречается разный формат хранения имен, в том числе существуют отличия для мужчин и женщин. Имя может быть составное (из нескольких имен) - рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Какое самое популярное женское имя на корабле? Какое самое популярное мужское имя на корабле? 

6)	Приведите список уникальных (неповторяющихся, только один человек имеет это имя) имен пассажиров, отдельно мужских и женских. Рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Выведите таблицу с этими пассажирами.

7)	Приведите множество всех имен пассажиров, отдельно мужских и женских. Рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена.

8)	Приведите список имен, которые одновременно встречаются у мужчин, и у женщин. Решите задачу, если рассматривать составные имена как одно имя, и если рассматривать составные имена отдельно. Выведите таблицу пассажирами, имеющими эти имена.

In [55]:
import re

In [149]:
df_titanic = pd.read_csv("titanic.csv")

#1:
passengers_count = df_titanic.shape[0]
survived_passengers = df_titanic[df_titanic["Survived"] == 1]
survived_passengers_count = survived_passengers.shape[0]
print(f"Доля выживших пассажиров: {survived_passengers_count / passengers_count}")

#2:
male_passengers = df_titanic[df_titanic["Sex"] == "male"]
male_passengers_count = male_passengers.shape[0]
female_passengers_count = passengers_count - male_passengers_count
print(f"Мужчин ехало на корабле: {male_passengers_count}")
print(f"Женщин ехало на корабле: {female_passengers_count}")
male_survivors = survived_passengers[survived_passengers["Sex"] == "male"]
male_survivors_count = male_survivors.shape[0]
female_survivors= survived_passengers[survived_passengers["Sex"] == "female"]
female_survivors_count = female_survivors.shape[0]
print(f"Доля выживших мужчин: {male_survivors_count / male_passengers_count}")
print(f"Доля выживших женщин: {female_survivors_count / female_passengers_count}")

#3:
first_class_passengers = df_titanic[df_titanic["Pclass"] == 1]
first_class_passengers_count = first_class_passengers.shape[0]
second_class_passengers = df_titanic[df_titanic["Pclass"] == 2]
second_class_passengers_count = second_class_passengers.shape[0]
third_class_passengers = df_titanic[df_titanic["Pclass"] == 3]
third_class_passengers_count = third_class_passengers.shape[0]
print(f"Доля пассажиров первого класса: {first_class_passengers_count / passengers_count}")
print(f"Доля пассажиров второго класса: {second_class_passengers_count / passengers_count}")
print(f"Доля пассажиров третьего класса: {third_class_passengers_count / passengers_count}")
first_class_survivors = survived_passengers[survived_passengers["Pclass"] == 1]
first_class_survivors_count = first_class_survivors.shape[0]
second_class_survivors = survived_passengers[survived_passengers["Pclass"] == 2]
second_class_survivors_count = second_class_survivors.shape[0]
third_class_survivors = survived_passengers[survived_passengers["Pclass"] == 3]
third_class_survivors_count = third_class_survivors.shape[0]
print(f"Доля выживших пассажиров первого класса: {first_class_survivors_count / first_class_passengers_count}")
print(f"Доля выживших пассажиров второго класса: {second_class_survivors_count / second_class_passengers_count}")
print(f"Доля выживших пассажиров третьего класса: {third_class_survivors_count / third_class_passengers_count}")

#4:
titanic_passengers_ages = df_titanic["Age"]
titanic_passengers_ages = titanic_passengers_ages.dropna()
print(f"Средний возраст пассажиров: {titanic_passengers_ages.agg('mean')}")
print(f"Медиана возраста пассажиров: {titanic_passengers_ages.agg('median')}")
survived_passengers_ages = survived_passengers["Age"]
survived_passengers_ages = survived_passengers_ages.dropna()
print(f"Средний возраст выживших пассажиров: {survived_passengers_ages.agg('mean')}")
print(f"Медиана возраста выживших пассажиров: {survived_passengers_ages.agg('median')}")
dead_passengers_ages = df_titanic[df_titanic["Survived"] == 0]["Age"]
dead_passengers_ages = dead_passengers_ages.dropna()
print(f"Средний возраст погибших пассажиров: {dead_passengers_ages.agg('mean')}")
print(f"Медиана возраста погибших пассажиров: {dead_passengers_ages.agg('median')}")

#5:
def get_not_compound_names(name_str : str, sex : str):
    true_names = []

    status_split = name_str.split(". ")
    before_status = status_split[0].split(" ")
    after_status = status_split[1]
    
    surname = before_status[0:len(before_status) - 1]
    status = before_status[len(before_status) - 1]
    
    all_person_names = re.findall(r"[(]+[\w\s\-\"]+[)]+|[\w]+[a-zA-Z\-\"]+[\w]+", after_status)

    if status == "Mrs":
        match = None
        names = []
        for name in all_person_names:
            if re.match(r"[(\"]+[\w\s\-\.]+[)\"]+", name):
                match = re.match(r"[(\"]+([\w\s\-\.]+)[)\"]+", name).group(1)
                break
            else:
                names.append(name)
        if match is not None:
            match_words = match.split(" ")
            for name in match_words[0:len(match_words)-1]:
                if len(name) > 2:
                    true_names.append(name)
        else:
            for name in names:
                if len(name) > 2:
                    true_names.append(name)
    else:
        for name in all_person_names:
            if re.match(r"[(\"]+[\w\s\-\.]+[)\"]+", name):
                match = re.match(r"[(\"]+([\w\s\-\.]+)[)\"]+", name).group(1)
                match_words = match.split(" ")
                if 'Mr' in match_words:
                    words = match_words[match.find('Mr')+1:len(match_words)-1]
                    for word in words:
                        if len(word) > 2:
                            true_names.append(word)
                elif 'Mrs' in match_words:
                    words = match_words[match.find('Mrs')+1:len(match_words)-1]
                    for word in words:
                        if len(word) > 2:
                            true_names.append(word)
                elif len(match_words) == 1:
                    word = match_words[0]
                    if len(word) > 2:
                        true_names.append(word)
                else:
                    words = match_words[0:len(match_words)-1]
                    for word in words:
                        if len(word) > 2:
                            true_names.append(word)
            else:
                true_names.append(name)
    print(name_str)
    print(true_names)
    return true_names
                    

for i in df_titanic.transpose():
    get_not_compound_names(df_titanic.transpose()[i]["Name"], df_titanic.transpose()[i]["Sex"])

Доля выживших пассажиров: 0.3838383838383838
Мужчин ехало на корабле: 577
Женщин ехало на корабле: 314
Доля выживших мужчин: 0.18890814558058924
Доля выживших женщин: 0.7420382165605095
Доля пассажиров первого класса: 0.24242424242424243
Доля пассажиров второго класса: 0.20650953984287318
Доля пассажиров третьего класса: 0.5510662177328844
Доля выживших пассажиров первого класса: 0.6296296296296297
Доля выживших пассажиров второго класса: 0.47282608695652173
Доля выживших пассажиров третьего класса: 0.24236252545824846
Средний возраст пассажиров: 29.69911764705882
Медиана возраста пассажиров: 28.0
Средний возраст выживших пассажиров: 28.343689655172415
Медиана возраста выживших пассажиров: 28.0
Средний возраст погибших пассажиров: 30.62617924528302
Медиана возраста погибших пассажиров: 28.0
Braund, Mr. Owen Harris
['Owen', 'Harris']
Cumings, Mrs. John Bradley (Florence Briggs Thayer)
['Florence', 'Briggs']
Heikkinen, Miss. Laina
['Laina']
Futrelle, Mrs. Jacques Heath (Lily May Peel)
['